In [25]:
import sklearn
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [16]:
import os
import sys
import pandas as pd, numpy as np
import timeit
import joblib
from copy import deepcopy
import pandas as pd
import time

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.python.client import device_lib
from sklearn.metrics import accuracy_score, roc_auc_score

import pathlib
sys.path.insert(0, os.getcwd().split('scripts')[0])

print(sys.version, sys.platform, sys.executable) #Displays what environment you are actually using.

from src import models
from src import sparse_soft_trees
from src import utils
from data import data_utils


from tensorflow.python.client import device_lib
def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def get_available_cpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'CPU']


os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

3.10.11 (main, Apr 20 2023, 19:02:41) [GCC 11.2.0] linux /home/gridsan/shibal/.conda/envs/MOETF29/bin/python


In [28]:
data = load_breast_cancer()
x = data.data
y = data.target

seed = 8
x_train_valid, x_test, y_train_valid, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=seed)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_valid, y_train_valid, test_size=0.2, stratify=y_train_valid, random_state=seed)

assert len(y.shape)==1
# print("Class-set (Train)", np.unique(data_processed.y_train_processed))
# print("Class-set (Valid)", np.unique(data_processed.y_valid_processed))
# print("Class-set (Train-Valid)", np.unique(data_processed.y_train_valid_processed))
# print("Class-set (Test)", np.unique(data_processed.y_test_processed))
num_classes = y.max(axis=0)+1
print("====================num_classes:", num_classes)
num_features = x.shape[1]
print("====================num_features:", num_features)

x_preprocessor = StandardScaler()
x_train_processed = x_preprocessor.fit_transform(x_train)
x_valid_processed = x_preprocessor.transform(x_valid)
x_test_processed = x_preprocessor.transform(x_test)

y_train_processed = y_train
y_valid_processed = y_valid
y_test_processed = y_test


====================num_classes: 2
====================num_features: 30


# Note 

Note that there are two configurations:
1. Fixed Group-L0 regularization:
use_annealing=False, kernel_constraint=varied across a range of params, number of epochs influence the degree of sparsity.
2. Anneals the Group-L0 regularization:
use_annealing=True, kernel_constraint=100, temperature = 0.01, epochs influence the degree of sparsity in the model

Below we show running the model with a particular choice of hyperparameters for case (2). 

Since we solve the unconstrained feature selection problem, we need to sweep these parameters to find settings where the model the satisfies a desired budget of features. See main_classification_public_data.py for hyperparameter tuning with optuna and SkinnyTrees-Results.ipynb can be used to compile results.

In [31]:
constant_batch_size = 16
batch_size_scaler = 1
batch_size = constant_batch_size*batch_size_scaler
constant_learning_rate = 0.1
early_stopping = False


num_train_samples = x_train.shape[0]
epochs = 1000

if num_train_samples % batch_size == 0:
    epoch_step = num_train_samples / batch_size
else:
    epoch_step = int(num_train_samples / batch_size) + 1

#         epochs = trial.suggest_int('epochs_new', 5, epochs)
print("==============No LR scheduler, Epochs:", epochs, "Batch-size:", batch_size)
print("==============epochs:", epochs)
learning_rate = constant_learning_rate
lr_schedule = utils.ConstantLearningRate(
    learning_rate
)
optim = tf.keras.optimizers.SGD(lr_schedule)

### Soft Decision Tree parameters 
num_trees = 20
depth = 4

activation = tf.keras.activations.sigmoid

use_annealing = True
kernel_constraint = 100
temperature = 0.01
kernel_l2 = 0.1
kernel_l2 = kernel_l2/(num_trees*(2**depth - 1))
kernel_regularizer = tf.keras.regularizers.L2(kernel_l2)
if use_annealing:
    kernel_constraint = kernel_constraint/num_features
    temperature = temperature
    kernel_constraint=sparse_soft_trees.ProximalGroupL0(lr=lr_schedule, lam=kernel_constraint, temperature=temperature, use_annealing=True, name='ProximalGroupL0')
else:
    kernel_constraint = kernel_constraint/num_features
    kernel_constraint=sparse_soft_trees.ProximalGroupL0(lr=lr_schedule, lam=kernel_constraint, use_annealing=False, name='ProximalGroupL0')
print("===========kernel_regularizer:", kernel_regularizer)
print("===========kernel_constraint:", kernel_constraint)


### Optimization parameters
leaf_dims = (num_classes, )
x = tf.keras.layers.Input(name='input', shape=x_train_processed.shape[1:])
submodel = models.create_model(
    x,
    num_trees,
    depth,
    leaf_dims,
    activation=activation,
    kernel_regularizer=kernel_regularizer,
    kernel_constraint=kernel_constraint,
)
x = submodel.input
outputs = submodel(x)
# print(outputs)
ypred = tf.keras.layers.Activation('linear')(outputs)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model = tf.keras.Model(inputs=x, outputs=ypred)

model.summary()

monitor = 'val_accuracy'
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optim, metrics=metrics)
cb = sparse_soft_trees.SparsityHistory()
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    cb
]    
if early_stopping:
    callbacks.append(
        sparse_soft_trees.EarlyStopping(
            config['cardinality'], monitor=monitor, patience=100, verbose=1, mode='auto', restore_best_weights=True
        ),
    )
# print("====================y.shape", data_processed.y_train_processed.shape)
if len(get_available_gpus())==0:
    history = model.fit(x=x_train_processed, 
              y=y_train_processed,
              epochs=epochs, 
              batch_size=batch_size, 
              shuffle=True,
              callbacks=callbacks,
              validation_data=(x_valid_processed, y_valid_processed),
              verbose=1, 
              )  
else:
    with tf.device(get_available_gpus()[0]):
        history = model.fit(x=x_train_processed, 
                  y=y_train_processed,
                  epochs=epochs, 
                  batch_size=batch_size, 
                  shuffle=True,
                  callbacks=callbacks,
                  validation_data=(x_valid_processed, y_valid_processed),
                  verbose=0, 
                  )  
number_of_epochs_it_ran = len(history.history['loss'])

val_loss_history = history.history['val_loss']

if early_stopping:
    best_epoch = callbacks[-1].best_epoch # np.argmin(val_loss_history) + 1
else:
    best_epoch = len(val_loss_history)
print("============best_epoch:", best_epoch)
print("============number_of_epochs_it_ran:", number_of_epochs_it_ran)

feature_sparsity_history = cb.selected_features[1:(best_epoch+1)]
approximate_feature_sparsity_history = cb.approximately_selected_features[1:(best_epoch+1)]

with tf.device(get_available_cpus()[0]):
    # Check for infinite loss
    training_loss = model.evaluate(x_train_processed,
                                   y_train_processed,
                                   batch_size=batch_size,
                                   verbose=0)

    if np.isfinite(np.sum(training_loss)) or ~np.isnan(np.sum(training_loss)):
        # Evaluation

        y_valid_pred = model.predict(x_valid_processed)
        y_valid_prob = tf.nn.softmax(y_valid_pred, axis=1).numpy()
        y_valid_pred_classes = np.argmax(y_valid_prob, axis=1)
        classes = np.shape(y_valid_prob)[1]
        if classes==2:
            y_valid_prob = tf.gather(y_valid_prob, indices=[1], axis=1).numpy()
        accuracy_valid = accuracy_score(y_valid_processed, y_valid_pred_classes)
        auc_valid = roc_auc_score(
            y_valid_processed,
            y_valid_prob,
            multi_class='ovo'
        )

        y_test_pred = model.predict(x_test_processed)
        y_test_prob = tf.nn.softmax(y_test_pred, axis=1).numpy()
        y_test_pred_classes = np.argmax(y_test_prob, axis=1)
        if classes==2:
            y_test_prob = tf.gather(y_test_prob, indices=[1], axis=1).numpy()
        accuracy_test = accuracy_score(y_test_processed, y_test_pred_classes)
        auc_test = roc_auc_score(
            y_test_processed,
            y_test_prob,
            multi_class='ovo'
        )
        print('accuracy (valid):', accuracy_valid)
        print('accuracy (test):', accuracy_test)
        print('auc (valid):', auc_valid)
        print('auc (test):', auc_test)
print("num_epochs", number_of_epochs_it_ran)
print("val_loss_history", val_loss_history)
print("feature_sparsity_history", feature_sparsity_history)    
print("feature_sparsity", feature_sparsity_history[-1])    


==============No LR scheduler, Epochs: 1000 Batch-size: 16
==============epochs: 1000
===========kernel_regularizer: <keras.regularizers.L2 object at 0x7f3d0c448b20>
===========kernel_constraint: <src.sparse_soft_trees.ProximalGroupL0 object at 0x7f3d0c448af0>
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 30)]              0         
                                                                 
 model_6 (Functional)        (None, 2)                 9940      
                                                                 
 activation_3 (Activation)   (None, 2)                 0         
                                                                 
Total params: 9,940
Trainable params: 9,940
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
23/23 [==============================] - 4s 29m

23/23 [==============================] - 0s 7ms/step - loss: 0.0926 - accuracy: 0.9835 - lam: 3.3273 - nnz: 30.0000 - group-l0-reg: 99.8196 - val_loss: 0.1244 - val_accuracy: 0.9670 - val_lam: 3.3280 - val_nnz: 30.0000 - val_group-l0-reg: 99.8404
Epoch 29/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0917 - accuracy: 0.9835 - lam: 3.3286 - nnz: 30.0000 - group-l0-reg: 99.8567 - val_loss: 0.1240 - val_accuracy: 0.9670 - val_lam: 3.3291 - val_nnz: 30.0000 - val_group-l0-reg: 99.8732
Epoch 30/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0908 - accuracy: 0.9835 - lam: 3.3295 - nnz: 30.0000 - group-l0-reg: 99.8861 - val_loss: 0.1236 - val_accuracy: 0.9670 - val_lam: 3.3300 - val_nnz: 30.0000 - val_group-l0-reg: 99.8992
Epoch 31/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0899 - accuracy: 0.9863 - lam: 3.3303 - nnz: 30.0000 - group-l0-reg: 99.9095 - val_loss: 0.1232 - val_accuracy: 0.9670 - val_lam: 3.3307 - val_nnz: 30.000

23/23 [==============================] - 0s 5ms/step - loss: 0.0760 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 99.9999 - val_loss: 0.1188 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 99.9999
Epoch 60/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0757 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 99.9999 - val_loss: 0.1187 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 99.9999
Epoch 61/1000
23/23 [==============================] - 0s 9ms/step - loss: 0.0754 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 99.9999 - val_loss: 0.1187 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 99.9999
Epoch 62/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0752 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 99.9999 - val_loss: 0.1186 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.000

23/23 [==============================] - 0s 6ms/step - loss: 0.0686 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1170 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 100.0000
Epoch 91/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0684 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1170 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 100.0000
Epoch 92/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0683 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1169 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 100.0000
Epoch 93/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0681 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1168 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz:

23/23 [==============================] - 0s 8ms/step - loss: 0.0637 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1157 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 100.0000
Epoch 122/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0636 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1157 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 100.0000
Epoch 123/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0635 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1157 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 30.0000 - val_group-l0-reg: 100.0000
Epoch 124/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0633 - accuracy: 0.9918 - lam: 3.3333 - nnz: 30.0000 - group-l0-reg: 100.0000 - val_loss: 0.1155 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_n

...

23/23 [==============================] - 0s 7ms/step - loss: 0.0265 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1241 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 866/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0265 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1242 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 867/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0264 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1242 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 868/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0264 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1242 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.

23/23 [==============================] - 0s 6ms/step - loss: 0.0259 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1251 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 897/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0259 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1252 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 898/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0259 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1253 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 899/1000
23/23 [==============================] - 0s 8ms/step - loss: 0.0259 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1253 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.

23/23 [==============================] - 0s 7ms/step - loss: 0.0254 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1262 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 928/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0254 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1262 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 929/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0254 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1263 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 930/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0254 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1263 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.

23/23 [==============================] - 0s 6ms/step - loss: 0.0249 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1269 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 959/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0249 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1270 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 960/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0249 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1271 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 961/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0249 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1272 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.

23/23 [==============================] - 0s 6ms/step - loss: 0.0245 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1278 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 990/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0245 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1279 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 991/1000
23/23 [==============================] - 0s 7ms/step - loss: 0.0244 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1280 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.0000 - val_group-l0-reg: 86.6667
Epoch 992/1000
23/23 [==============================] - 0s 6ms/step - loss: 0.0244 - accuracy: 1.0000 - lam: 3.3333 - nnz: 26.0000 - group-l0-reg: 86.6666 - val_loss: 0.1280 - val_accuracy: 0.9670 - val_lam: 3.3333 - val_nnz: 26.